In [ ]:
from gurobipy import GRB
import gurobipy as gp
import numpy as np

def modified_dfs(G_list, visited, src, dest, temp):
  if src == dest:
    print(temp)
    res.append(temp[:])
    return

  visited[src] = True

  for i in G_list[src]:
    if not visited[i]:
      temp.append(i)
      modified_dfs(G_list, visited, i, dest, temp)
      temp.pop()

  visited[src] = False

# Input của Congestion Minimization
G = np.array([[0, 1, 0, 0, 0],
              [0, 0, 1, 1, 0],
              [0, 0, 0, 1, 0],
              [0, 0, 0, 0, 1],
              [0, 1, 0, 0, 0]])
path = [[4, 3], [1, 4], [0, 2], [2, 1]]

n = G.shape[0]
e = np.count_nonzero(G) # e = np.count_nonzero(G == 1)

# Tạo index cho cạnh trong G
dict = {(src, dest): i for i, (src, dest) in enumerate(zip(*np.where(G == 1)))}
print(dict)

# Adjacent list cho đồ thị G
G_list = [[] for _ in range(n)]
for i in range(n):
  for j in range(n):
    if G[i][j] == 1:
      G_list[i].append(j)

global res
res = []

num_of_path = []
matrix_res = []

for x in path:
  temp = [x[0]]
  modified_dfs(G_list, np.zeros(n, dtype = bool), x[0], x[1], temp)
  num_of_path.append(len(res))

for i in range(len(num_of_path) - 1, 0, -1):
  num_of_path[i] = num_of_path[i] - num_of_path[i - 1]

for x in res:
  matrix_res.append([0 for _ in range(e)])
  for i, (src, dest) in enumerate(zip(x[:-1], x[1:])):
    matrix_res[-1][dict[(src, dest)]] = 1

matrix_res = np.array(list(map(list, zip(*matrix_res))))
print(matrix_res)

# Create a new model
model = gp.Model()

# Create variables
x = model.addMVar(matrix_res.shape[1], vtype = 'B', name = "x")
C = model.addVar(vtype = 'I', name = "C")

# Set objective function
model.setObjective(C, gp.GRB.MINIMIZE)

# Add constraints
for j in range(len(num_of_path)):
  model.addConstr(gp.quicksum(x[i] for i in range(sum(num_of_path[:j]) if j > 0 else 0, sum(num_of_path[:j + 1]))) == 1)
for i in range(e):
  model.addConstr(matrix_res[i, :] @ x <= C)

# Solve it!
model.optimize()

print("\n-------My solution-------")
print(f"Optimal objective value: {model.objVal}")
print(f"Solution values: x = {x.X.astype(int)}")

: 